In [ ]:
import os

# Define the dataset path
dataset_path = '/kaggle/input/diamos-plant-dataset/Pear/leaves'

# Check if the dataset path exists
if not os.path.exists(dataset_path):
    print(f"Error: Dataset path '{dataset_path}' does not exist.")
else:
    # Count the number of files in each subfolder (Pear and leaves)
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path):
            num_files = len(os.listdir(folder_path))
            print(f"Number of files in '{folder}' folder: {num_files}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image

In [ ]:
# Define the path to your dataset
dataset_path = '/kaggle/input/diamos-plant-dataset/Pear/leaves'

In [ ]:
# Define transformations for preprocessing
transform = transforms.Compose([
transforms.Resize((224, 224)), # Resize image to 224x224
transforms.ToTensor(), # Convert image to PyTorch tensor
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]) # Normalize image

In [ ]:
# Function to check if image is valid
def is_valid_image(image_path):
    try:
        # Attempt to open and validate the image
        Image.open(image_path).convert('RGB')
        return True
    except (FileNotFoundError, OSError, ValueError):
        return False

In [ ]:
# Custom dataset class with error handling
class CustomImageFolder(ImageFolder):
    def __init__(self, root, transform=None):
        super().__init__(root, transform=transform)

    def __getitem__(self, index):
        # Get image path and label from original ImageFolder
        path, target = self.samples[index]

        # Check if image is valid
        if not is_valid_image(path):
            # Handle invalid image (e.g., return a placeholder image or skip)
            # For simplicity, returning a placeholder image and an out-of-bounds target
            invalid_image = torch.zeros((3, 224, 224))  # Placeholder image tensor
            return invalid_image, len(self.classes)  # Return an out-of-bounds target

        # Load and transform the image
        image = self.loader(path)
        if self.transform is not None:
            image = self.transform(image)

        return image, target

In [ ]:
# Load the custom dataset
dataset = CustomImageFolder(dataset_path, transform=transform)

# Define the DataLoader
batch_size = 32
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Define the custom CNN model
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)  # Adjust input size based on your image dimensions after pooling
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 32 * 56 * 56)  # Adjust based on your image dimensions after pooling
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Initialize the model
model = CustomCNN(num_classes=len(dataset.classes))

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=len(dataset.classes)) # Ignore the out-of-bounds target
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(data_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(data_loader)}], Loss: {running_loss / 10:.4f}')
            running_loss = 0.0

print('Finished Training.')

# Save the trained model if needed
torch.save(model.state_dict(), 'custom_cnn.pth')